# NYT API Exploration Project


## Mike Jacobs

###### MIS 680 Summer 2016

## Introduction
I decided to do my project on the New York Times (NYT) APIs:
http://developer.nytimes.com/
There are actually 13 different APIs available.  I decided to focus on the Article Search API and the Semantic API.
The NYT APIs return data in JSON format.  The article data goes back to 1851.

I don't have a specific plan for exactly what I want to accomplish in this project.  In general, I want to explore the NYT article data starting small and then expanding into more complex analysis.

A few ideas I have are:
- explore trends in articles over certain date ranges (keywords, language, etc)
- Perform sentiment analysis for certain topics in articles or in general
- Textual analysis (using NLTK)
- Stretch goal: Can I come up with some kind of prediction model using NYT article data?  What would I be predicting???

Some programmatic concerns and goals to be addressed:

Reading data
- Use of the New York Times article package to read data: https://pypi.python.org/pypi/nytimesarticle/0.1.0
(author Evan Sherlock)
- Other means to build queries to get data
- Specific query vs full data dump and python/R to analyze data
- API search limitatons: 1000 calls per day, 5 calls per second
- Use of looping and functions to process through data queries

Data structures
- What can I do with JSON files?
- convert to Dataframes?
- what kind of data cleaning will I have to do?

What is the NLTK package and what can I do with it?

How can I do sentiment analysis on NYT articles?

Why did some of the text above turn blue? #Note: it's no longer blue.  Think it had something to do with bulleted list

Ok, enough stalling - let's do something!

In [1]:
# NYT API key (I requested this.  My key is the same for both the artcle and semantic APIs)
api_key = 'PUT YOUR KEY HERE'

Import statements that I think I need for now

In [2]:
#imports
import json
import requests
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import math
from pandas import Series, DataFrame
import nltk
#import urllib3 #didn't use this
import simplejson as json

In [3]:
import urllib.request #used for connecting to NYtimes API for data read

In [4]:
import time #used to put .5 second delay in API data call

In [5]:
from itertools import chain #used to convert dictionary to dataframe

In [6]:
# in case I do some plots
%matplotlib inline

In [7]:
#New York Times article package
from nytimesarticle import articleAPI

In [8]:
# use my key with this package
api = articleAPI(api_key)

Ok, I'm ready for a first search.  Using the NYT article search package, I can specify things like:
- search criteria
- fields to search in (headline, body, etc)
- date range of articles
- page (each page is 10 articles)
- fields to return
The NYT API also has something called facet fields.  According to the NYT API readme (http://developer.nytimes.com/article_search_v2.json#/README):

"Use facets to view the relative importance of certain fields to a search term, and gain insight about how to best refine your queries and filter your search results.

The following fields can be used as facet fields: source, section_name, document_type, type_of_material and day_of_week.

Specify facets using the facet_fields parameter. The response will contain an array with a count for the five terms that have the highest count for each facet."

That sounds useful, but I'm not going to use it until I can get some basic searches working.

For this first search I'm just going look at articles from the last 2 weeks.  
First I actually tried just using a search string (like NRA), but that yielded 0 results.  I also tried the example from the documentation of the package ('Obama') and that didn't work either.  I'm wondering if something with this package doesn't work with the latest version fo the NYT API?  I spent about 2 hours trying to figure it out, but now I'm moving on.
my original code was articles = api.search(q = 'NRA')

In [19]:
#first search
articles = api.search(begin_date = 20160615)

That seemed to work - at least there weren't any errors.  Lets see what I got in terms of data and format.  

In [20]:
type(articles)

dict

In [21]:
len(articles)

3

In [22]:
articles

{'copyright': 'Copyright (c) 2013 The New York Times Company.  All Rights Reserved.',
 'response': {'docs': [{'_id': '576af55c38f0d83413c93cdf',
    'abstract': None,
    'blog': [],
    'byline': {'original': 'By TAS TOBEY',
     'person': [{'firstname': 'Tas',
       'lastname': 'TOBEY',
       'organization': '',
       'rank': 1,
       'role': 'reported'}]},
    'document_type': 'article',
    'headline': {'main': 'Three’s a Trend: Men’s Western Shirts'},
    'keywords': [{'is_major': 'N',
      'name': 'organizations',
      'rank': '1',
      'value': 'Topman (Fashion Label)'},
     {'is_major': 'N',
      'name': 'subject',
      'rank': '2',
      'value': 'Fashion and Apparel'},
     {'is_major': 'N',
      'name': 'subject',
      'rank': '4',
      'value': 'London Fashion Week'},
     {'is_major': 'N',
      'name': 'subject',
      'rank': '5',
      'value': 'Milan Fashion Week'},
     {'is_major': 'N',
      'name': 'subject',
      'rank': '6',
      'value': "Men's Fa

Ok, I got some results!  This package spits out results as a dictionary.  I'm more comfortable using a dataframe for analysis, so let's see if I can convert this to a dataframe.  Looking at the dictionary - it looks like the article results are in the 'docs' dictionary which is itself in the'response' dictionary.
Also, even though there are 9749 hits, I only got 10 articles (the first page).

In [23]:
articles.keys()

dict_keys(['copyright', 'response', 'status'])

In [24]:
articles['response']['docs']

[{'_id': '576af55c38f0d83413c93cdf',
  'abstract': None,
  'blog': [],
  'byline': {'original': 'By TAS TOBEY',
   'person': [{'firstname': 'Tas',
     'lastname': 'TOBEY',
     'organization': '',
     'rank': 1,
     'role': 'reported'}]},
  'document_type': 'article',
  'headline': {'main': 'Three’s a Trend: Men’s Western Shirts'},
  'keywords': [{'is_major': 'N',
    'name': 'organizations',
    'rank': '1',
    'value': 'Topman (Fashion Label)'},
   {'is_major': 'N',
    'name': 'subject',
    'rank': '2',
    'value': 'Fashion and Apparel'},
   {'is_major': 'N',
    'name': 'subject',
    'rank': '4',
    'value': 'London Fashion Week'},
   {'is_major': 'N',
    'name': 'subject',
    'rank': '5',
    'value': 'Milan Fashion Week'},
   {'is_major': 'N',
    'name': 'subject',
    'rank': '6',
    'value': "Men's Fashion Week"}],
  'lead_paragraph': 'Topman Design, Cmmn Swdn and Missoni all showed modern versions of the classic American style.',
  'multimedia': [{'height': 126,
  

In [25]:
articledata = pd.DataFrame(articles['response']['docs'])

In [26]:
articledata

_id abstract blog  \
0  576af55c38f0d83413c93cdf     None   []   
1  5761930938f0d81a0f68f3e0     None   []   
2  575b3d8438f0d82d6b874756     None   []   
3  5771867538f0d82529b0b360     None   []   
4  5771865638f0d82529b0b35e     None   []   
5  576d988c38f0d87405ee5e89     None   []   
6  576d4a8138f0d8520fd89355     None   []   
7  5771584e38f0d82529b0b2cb     None   []   
8  576d6c0138f0d87405ee5e06     None   []   
9  576d5a6e38f0d87405ee5da9     None   []   

                                              byline document_type  \
0  {'original': 'By TAS TOBEY', 'person': [{'firs...       article   
1  {'original': 'By AIMEE FARRELL', 'person': [{'...       article   
2  {'original': 'By THE NEW YORK TIMES MAGAZINE',...       article   
3  {'original': 'By BARRY FRIEDMAN', 'person': [{...       article   
4  {'original': 'By DANI SHAPIRO', 'person': [{'f...       article   
5  {'original': 'By MELENA RYZIK', 'person': [{'f...       article   
6  {'original': 'By GREGORY COWLES', 'person': [{...       article   
7  {'original': 'Tejal Rao', 'person': [{'firstna...        recipe   
8  {'original': 'Melissa Clark', 'person': [{'fir...        recipe   
9  {'original': 'David Tanis', 'person': [{'first...        recipe   

                                            headline  \
0  {'main': 'Three’s a Trend: Men’s Western Shirts'}   
1  {'main': 'How Duro Olowu Gets Dressed in the M...   
2                           {'main': 'Man on Spire'}   
3  {'content_kicker': 'Nonfiction', 'kicker': 'No...   
4  {'content_kicker': 'Author’s Note', 'kicker': ...   
5  {'content_kicker': 'Snapshot', 'kicker': 'Snap...   
6                        {'main': 'Inside the List'}   
7  {'name': 'Sweet Mochi With Red Bean Filling', ...   
8  {'name': 'Buckwheat Berry Striped Cake', 'main...   
9  {'name': 'Harira Soup', 'main': 'A Cinnamon-Sc...   

                                            keywords  \
0  [{'name': 'organizations', 'value': 'Topman (F...   
1  [{'name': 'subject', 'value': 'Fashion and App...   
2  [{'name': 'subject', 'value': 'Virtual Reality...   
3  [{'name': 'subject', 'value': 'Books and Liter...   
4  [{'name': 'subject', 'value': 'Books and Liter...   
5  [{'name': 'subject', 'value': 'Country Music',...   
6  [{'name': 'subject', 'value': 'Books and Liter...   
7                                                 []   
8                                                 []   
9                                                 []   

                                      lead_paragraph  \
0  Topman Design, Cmmn Swdn and Missoni all showe...   
1  For the fashion designer and curator of the ne...   
2  Climb to the top of 1 World Trade Center in vi...   
3  Two books offer opposing arguments on crime an...   
4  After my second memoir, people stopped asking ...   
5  Ms. Parton takes a break from her 60-city tour...   
6  Bobby Brown’s memoir, at No. 9 on the hardcove...   
7  Nothing compares to the texture of fresh mochi...   
8  A combination of buckwheat and whole wheat flo...   
9  Harira, a savory Moroccan soup made with dried...   

                                          multimedia     news_desk print_page  \
0  [{'width': 190, 'height': 126, 'type': 'image'...        TStyle       None   
1  [{'width': 190, 'height': 126, 'type': 'image'...        TStyle          3   
2  [{'width': 190, 'height': 126, 'type': 'image'...      Magazine       None   
3  [{'width': 190, 'height': 126, 'type': 'image'...    BookReview          8   
4  [{'width': 190, 'height': 126, 'type': 'image'...    BookReview         25   
5  [{'width': 190, 'height': 126, 'type': 'image'...  Arts&Leisure       None   
6  [{'width': 190, 'height': 126, 'type': 'image'...    BookReview         22   
7  [{'width': 75, 'height': 75, 'credit': 'Evan S...          None       None   
8  [{'width': 75, 'height': 75, 'credit': 'Andrew...          None       None   
9  [{'width': 75, 'height': 75, 'credit': 'Evan S...          None       None   

              

In [29]:
articledata.head()

_id abstract blog  \
0  576af55c38f0d83413c93cdf     None   []   
1  5761930938f0d81a0f68f3e0     None   []   
2  575b3d8438f0d82d6b874756     None   []   
3  576d988c38f0d87405ee5e89     None   []   
4  576d4a8138f0d8520fd89355     None   []   

                                              byline document_type  \
0  {'person': [{'organization': '', 'firstname': ...       article   
1  {'person': [{'organization': '', 'firstname': ...       article   
2  {'person': [], 'organization': 'THE NEW YORK T...       article   
3  {'person': [{'organization': '', 'firstname': ...       article   
4  {'person': [{'organization': '', 'firstname': ...       article   

                                            headline  \
0  {'main': 'Three’s a Trend: Men’s Western Shirts'}   
1  {'main': 'How Duro Olowu Gets Dressed in the M...   
2                           {'main': 'Man on Spire'}   
3  {'main': 'Dolly Parton Is Proud of Her Gay Fan...   
4                        {'main': 'Inside the List'}   

                                            keywords  \
0  [{'value': 'Topman (Fashion Label)', 'name': '...   
1  [{'value': 'Fashion and Apparel', 'name': 'sub...   
2  [{'value': 'Virtual Reality (Computers)', 'nam...   
3  [{'value': 'Country Music', 'name': 'subject',...   
4  [{'value': 'Books and Literature', 'name': 'su...   

                                      lead_paragraph  \
0  Topman Design, Cmmn Swdn and Missoni all showe...   
1  For the fashion designer and curator of the ne...   
2  Climb to the top of 1 World Trade Center in vi...   
3  Ms. Parton takes a break from her 60-city tour...   
4  Bobby Brown’s memoir, at No. 9 on the hardcove...   

                                          multimedia     news_desk print_page  \
0  [{'height': 126, 'type': 'image', 'width': 190...        TStyle       None   
1  [{'height': 126, 'type': 'image', 'width': 190...        TStyle          3   
2  [{'height': 126, 'type': 'image', 'width': 190...      Magazine       None   
3  [{'height': 126, 'type': 'image', 'width': 190...  Arts&Leisure       None   
4  [{'height': 126, 'type': 'image', 'width': 190...    BookReview         22   

               pub_date section_name slideshow_credits  \
0  2016-07-22T00:00:00Z   T Magazine              None   
1  2016-07-15T00:00:00Z   T Magazine              None   
2  2016-07-05T00:00:00Z     Magazine              None   
3  2016-07-03T00:00:00Z         Arts              None   
4  2016-07-03T00:00:00Z        Books              None   

                                             snippet              source  \
0  Topman Design, Cmmn Swdn and Missoni all showe...  The New York Times   
1  For the fashion designer and curator of the ne...  The New York Times   
2  Climb to the top of 1 World Trade Center in vi...  The New York Times   
3  Ms. Parton takes a break from her 60-city tour...  The New York Times   
4  Bobby Brown’s memoir, at No. 9 on the hardcove...  The New York Times   

    subsection_name type_of_material  \
0  Fashion & Beauty             News   
1  Fashion & Beauty             News   
2              None             News   
3             Music             News   
4       Book Review             News   

                                             web_url word_count  
0  http://www.nytimes.com/2016/07/22/t-magazine/f...         96  
1  http://www.nytimes.com/2016/06/15/t-magazine/f...        650  
2  http://www.nytimes.com/2016/07/05/magazine/man...        200  
3  http://www.nytimes.com/2016/07/03/arts/music/d...        712  
4  http://www.nytimes.com/2016/07/03/books/review...        517

In [27]:
articledata.tail()

_id abstract blog  \
5  576d988c38f0d87405ee5e89     None   []   
6  576d4a8138f0d8520fd89355     None   []   
7  5771584e38f0d82529b0b2cb     None   []   
8  576d6c0138f0d87405ee5e06     None   []   
9  576d5a6e38f0d87405ee5da9     None   []   

                                              byline document_type  \
5  {'original': 'By MELENA RYZIK', 'person': [{'f...       article   
6  {'original': 'By GREGORY COWLES', 'person': [{...       article   
7  {'original': 'Tejal Rao', 'person': [{'firstna...        recipe   
8  {'original': 'Melissa Clark', 'person': [{'fir...        recipe   
9  {'original': 'David Tanis', 'person': [{'first...        recipe   

                                            headline  \
5  {'content_kicker': 'Snapshot', 'kicker': 'Snap...   
6                        {'main': 'Inside the List'}   
7  {'name': 'Sweet Mochi With Red Bean Filling', ...   
8  {'name': 'Buckwheat Berry Striped Cake', 'main...   
9  {'name': 'Harira Soup', 'main': 'A Cinnamon-Sc...   

                                            keywords  \
5  [{'name': 'subject', 'value': 'Country Music',...   
6  [{'name': 'subject', 'value': 'Books and Liter...   
7                                                 []   
8                                                 []   
9                                                 []   

                                      lead_paragraph  \
5  Ms. Parton takes a break from her 60-city tour...   
6  Bobby Brown’s memoir, at No. 9 on the hardcove...   
7  Nothing compares to the texture of fresh mochi...   
8  A combination of buckwheat and whole wheat flo...   
9  Harira, a savory Moroccan soup made with dried...   

                                          multimedia     news_desk print_page  \
5  [{'width': 190, 'height': 126, 'type': 'image'...  Arts&Leisure       None   
6  [{'width': 190, 'height': 126, 'type': 'image'...    BookReview         22   
7  [{'width': 75, 'height': 75, 'credit': 'Evan S...          None       None   
8  [{'width': 75, 'height': 75, 'credit': 'Andrew...          None       None   
9  [{'width': 75, 'height': 75, 'credit': 'Evan S...          None       None   

               pub_date section_name slideshow_credits  \
5  2016-07-03T00:00:00Z         Arts              None   
6  2016-07-03T00:00:00Z        Books              None   
7  2016-06-29T00:00:00Z         None              None   
8  2016-06-29T00:00:00Z         None              None   
9  2016-06-29T00:00:00Z         None              None   

                                             snippet              source  \
5  Ms. Parton takes a break from her 60-city tour...  The New York Times   
6  Bobby Brown’s memoir, at No. 9 on the hardcove...  The New York Times   
7  Nothing compares to the texture of fresh mochi...           du_recipe   
8  A combination of buckwheat and whole wheat flo...           du_recipe   
9  Harira, a savory Moroccan soup made with dried...           du_recipe   

  subsection_name type_of_material  \
5           Music             News   
6     Book Review             News   
7            None           Recipe   
8            None           Recipe   
9            None           Recipe   

                                             web_url word_count  
5  http://www.nytimes.com/2016/07/03/arts/music/d...        691  
6  http://www.nytimes.com/2016/07/03/books/review...        517  
7  http://cooking.nytimes.com/recipes/1018195-swe...        416  
8  http://cooking.nytimes.com/recipes/1018189-buc...        184  
9  http://cooking.nytimes.com/recipes/1018190-har...        221

So I have a dataframe, but there are some issues:
- the article pubdates have T00:00:00Z at the end
- many of the columns still have dictionaries inside them
- some nulls
- this only returned the first page of results - so only 10 articles

I need to do some data cleaning before using this.
I also think I'll need some kind of loop to get more than 1 page of results.

I'm going to start over and use a loop to query the api just using url calls.
Hopefully, this isn't cheating, but I found a site where someone did this for the NYT API:
http://www.aaronschlegel.com/notebook/store-new-york-times-api-data-in-amazon-s3-with-python/

NOTE:  I had to make changes to the code I borrowed:
-Urllib2 seems to not work with python 3 so I used urllib.request

-changed 'offset' to 'page' -- this one was really messing me up.
After getting into analysis, I realized that my dataframe of 1000 record was really only the same 10 records repeating over and over!

I'm going to attempt to use that code with some different queries.
The goal will be to get some article dataframes that I can use for further analysis.

The first step is to build a url with the query.
Now I have to strategize about how I'm going to pull in the data.  I could do some smaller more focused queries on different topics, or try to build one large query to pull in everything and then parse through it in various ways once I get it into a dataframe.  I'm going to try the large query - I'm limited by the 1000 calls per day limit from the API.  I'm using the API console (http://developer.nytimes.com/article_search_v2.json#/Console/GET/articlesearch.json) to test out queries to get estimates of the results.  So I think if I can limit my query to 10,000 results (1000 pages) then I should be ok.
If I planned ahead I could have built my result set over a number of weeks with a query every day.  I can't think of another way to get more data?
So much for my dream of building a dataframe of every NYT article going back to 1850...

In [9]:
#begining of the search string and json output
nyt_api = 'http://api.nytimes.com/svc/search/v2/articlesearch.'
out_format = 'json'

In [9]:
#then add the biggest date range that yields <10,000 results
#trying to get everything from 2016, but have to restrict to 2 weeks in June 2016
begin_date = '20160615'
end_date = '20160626'
date = 'begin_date=' + begin_date + '&' + 'end_date=' + end_date + '&'

In [10]:
#build the url query.  
offset = 0 #this brings all pages
url = nyt_api + out_format + '?' + date + 'page=' + str(offset) + '&' + 'api-key=' + api_key

In [11]:
#set pages to max, I'm using slightly less than 1000 since I made some calls already today.  
#I'm going to test out a larger query tomorrow - maybe I'm wrong about the 1000 page max?
pages = 100 # see note below about why pages set to only 100

In [12]:
## This section is commented out because it takes about 10 minutes to run.
## This is the API data call loop.  I subsequently stored the results of this into a dataframe and saved as CSV
## Then I just loaded that CSV for doing analysis and data manipulation.

# NOTE:  I changed this to only get 10 pages (1000 records).  After dealing with the repeating records problem,
# I then locked myself out of the API due to too many calls for a day.  So I limited myself to only 1000 records
# in case I had more problems.  Would have liked to be using more data.

#loop for the query 
#dicts = []
#for i in range(0, pages):
#    offset = i #for page number
#    url = nyt_api + out_format + '?' + date + 'page=' + str(offset) + '&' + 'api-key=' + api_key
#    #response = urllib2.urlopen(url).read() #urllib2 didn't work - I think it's for the previous python version
#    response = urllib.request.urlopen(url).read()
#    #response = requests.get(url).text #this was my backup incase urllib didn't work
#    dict_response = json.loads(response, encoding = 'utf-8') #dictionary of API call response
#    response = dict_response['response'] 
#    docs = response['docs'] #get just the articles
#    dicts.append(docs) #append each API call
#    time.sleep(0.5) # time delay

In [13]:
#len(dicts)

100

In [14]:
#create a dataframe
# NOTE: commented this out because data is loaded below from CSV
#data = pd.DataFrame(list(chain.from_iterable(dicts)))

In [16]:
#data.head(7)

_id abstract blog  \
0  5770a4e538f0d8170405490b     None   []   
1  5770a44438f0d8170405490a     None   []   
2  5770a2e438f0d81704054909     None   []   
3  5770a00a38f0d81704054907     None   []   
4  57709b3938f0d81704054905     None   []   
5  577099a538f0d81704054904     None   []   
6  5770986838f0d817040548ff     None   []   

                                              byline document_type  \
0  {'original': 'By REUTERS', 'person': [], 'orga...       article   
1  {'original': 'By REUTERS', 'person': [], 'orga...       article   
2  {'original': 'By THE ASSOCIATED PRESS', 'perso...       article   
3  {'original': 'By REUTERS', 'person': [], 'orga...       article   
4  {'original': 'By THE ASSOCIATED PRESS', 'perso...       article   
5  {'original': 'By REUTERS', 'person': [], 'orga...       article   
6  {'original': 'By THE ASSOCIATED PRESS', 'perso...       article   

                                            headline keywords  \
0  {'print_headline': 'Bank of Japan Can Stand Pa...       []   
1  {'print_headline': 'Japan PM Campaigns on 'Sta...       []   
2  {'print_headline': 'Police: Man Fatally Shoots...       []   
3  {'print_headline': 'Suicide Bombers Kill Six P...       []   
4  {'print_headline': 'Simone Biles Glides to 4th...       []   
5  {'print_headline': 'Factbox: Copa America Winn...       []   
6  {'print_headline': 'Kuhl Beats Kershaw in MLB ...       []   

                                      lead_paragraph multimedia news_desk  \
0  The Bank of Japan does not need to expand mone...         []      None   
1  Japanese Prime Minister Shinzo Abe, stumping f...         []      None   
2  Police say they've arrested a man who fatally ...         []      None   
3  A number of suicide bombers killed six people ...         []      None   
4  The national titles are starting to run togeth...         []      None   
5  Winners of the Copa America since 1993 when it...         []      None   
6  Chad Kuhl seemingly didn't stand much of a cha...         []      None   

  print_page              pub_date  section_name slideshow_credits  \
0       None  2016-06-26T23:59:31Z  Business Day              None   
1       None  2016-06-26T23:54:41Z         World              None   
2       None  2016-06-26T23:50:30Z          U.S.              None   
3       None  2016-06-26T23:36:57Z         World              None   
4       None  2016-06-26T23:17:13Z        Sports              None   
5       None  2016-06-26T23:10:45Z        Sports              None   
6       None  2016-06-26T23:05:39Z        Sports              None   

                                             snippet   source subsection_name  \
0  The Bank of Japan does not need to expand mone...  Reuters            None   
1  Japanese Prime Minister Shinzo Abe, stumping f...  Reuters    Asia Pacific   
2  Police say they've arrested a man who fatally ...       AP            None   
3  A number of suicide bombers killed six people ...  Reuters     Middle East   
4  The national titles are starting to run togeth...       AP        Olympics   
5  Winners of the Copa America since 1993 when it...  Reuters          Soccer   
6  Chad Kuhl seemingly didn't stand much of a cha...       AP        Baseball   

  type_of_material                                            web_url  \
0             News  http://www.nytimes.com/reuters/2016/06/26/busi...   
1             News  http://www.nytimes.com/reuters/2016/06/26/worl...   
2             News  http://www.nytimes.com/aponline/2016/06/26/us/...   
3             News  http://www.nytimes.com/reuters/2016/06/26/worl...   
4             News  http://www.nytimes.com/aponline/2016/06/26/spo...   
5             News  http://www.nytimes.com/reuters/2016/06/26/spor...   
6             News  http://www.nytimes.com/aponline/2016/06/26/spo...   

  word_count  
0        159  
1        425  
2         79  
3         71  
4        850  
5         99  
6        801

In [17]:
#data.tail(7)

_id abstract blog  \
993  576e226a38f0d87405ee5f90     None   []   
994  576e205e38f0d87405ee5f8f     None   []   
995  576e1ecd38f0d87405ee5f8e     None   []   
996  576e1e2238f0d87405ee5f8d     None   []   
997  576e1cc238f0d87405ee5f8c     None   []   
998  576e1b4e38f0d87405ee5f8b     None   []   
999  576e1aa138f0d87405ee5f8a     None   []   

                                                byline document_type  \
993  {'original': 'By THE ASSOCIATED PRESS', 'perso...       article   
994  {'original': 'By THE ASSOCIATED PRESS', 'perso...       article   
995  {'original': 'By THE ASSOCIATED PRESS', 'perso...       article   
996  {'original': 'By THE ASSOCIATED PRESS', 'perso...       article   
997  {'original': 'By THE ASSOCIATED PRESS', 'perso...       article   
998  {'original': 'By THE ASSOCIATED PRESS', 'perso...       article   
999  {'original': 'By THE ASSOCIATED PRESS', 'perso...       article   

                                              headline keywords  \
993  {'print_headline': 'Doubling Up: Boudia Keeps ...       []   
994  {'print_headline': 'Bernie Worrell, Masterful ...       []   
995  {'print_headline': 'China Cuts Contact With Ta...       []   
996  {'print_headline': 'Extra Security Moves Honor...       []   
997  {'print_headline': 'APNewsBreak: Scientists Se...       []   
998  {'print_headline': 'Resurgent Peavy Leads Gian...       []   
999  {'print_headline': 'Pope Visits Armenia's Clos...       []   

                                        lead_paragraph multimedia news_desk  \
993  David Boudia could spend this summer doubling ...         []      None   
994  Bernie Worrell, the ingenious "Wizard of Woo" ...         []      None   
995  Beijing said Saturday it had cut off contact w...         []      None   
996  The stepped-up security surrounding San Franci...         []      None   
997  As the largest international gathering of cora...         []      None   
998  Jake Peavy threw seven strong innings in anoth...         []      None   
999  Pope Francis is wrapping up his trip to Armeni...         []      None   

    print_page              pub_date section_name slideshow_credits  \
993       None  2016-06-25T02:17:40Z       Sports              None   
994       None  2016-06-25T02:08:00Z         U.S.              None   
995       None  2016-06-25T02:01:07Z        World              None   
996       None  2016-06-25T01:56:52Z         U.S.              None   
997       None  2016-06-25T01:54:02Z         U.S.              None   
998       None  2016-06-25T01:44:57Z       Sports              None   
999       None  2016-06-25T01:42:15Z        World              None   

                                               snippet source subsection_name  \
993  David Boudia could spend this summer doubling ...     AP        Olympics   
994  Bernie Worrell, the ingenious "Wizard of Woo" ...     AP            None   
995  Beijing said Saturday it had cut off contact w...     AP    Asia Pacific   
996  The stepped-up security surrounding San Franci...     AP            None   
997  As the largest international gathering of cora...     AP            None   
998  Jake Peavy threw seven strong innings in anoth...     AP        Baseball   
999  Pope Francis is wrapping up his trip to Armeni...     AP          Europe   

    type_of_material                                            web_url  \
993             News  http://www.nytimes.com/aponline/2016/06/25/spo...   
994             News  http://www.nytimes.com/aponline/2016/06/25/us/...   
995             News  http://www.nytimes.com/aponline/2016/06/25/wor...   
996             News  http://www.nytimes.com/aponline/2016/06/25/us/...   
997             News  http://www.nytimes.com/aponline/2016/06/25/us/...   
998             News  http://www.nytimes.com/aponline/2016/06/25/spo...   
999             News  http://www.nytimes.com/aponline/2016/06/25/wor...   

    word_count  
993        735  
994        734  
995        454  
996        136 

In [18]:
#clean out the'T00:00:00Z' from pub date
#data['pub_date'] = data['pub_date'].str.replace('T00:00:00Z','')

Going to export this to a CSV before I do anything else

In [19]:
#commented this out since csv file already created
#data.to_csv('nytimes06262016.csv', sep=',', encoding='utf-8')

In [20]:
#data.info()
#data.shape

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 0 to 999
Data columns (total 20 columns):
_id                  1000 non-null object
abstract             13 non-null object
blog                 1000 non-null object
byline               933 non-null object
document_type        1000 non-null object
headline             1000 non-null object
keywords             1000 non-null object
lead_paragraph       993 non-null object
multimedia           1000 non-null object
news_desk            996 non-null object
print_page           230 non-null object
pub_date             1000 non-null object
section_name         996 non-null object
slideshow_credits    17 non-null object
snippet              1000 non-null object
source               1000 non-null object
subsection_name      561 non-null object
type_of_material     1000 non-null object
web_url              1000 non-null object
word_count           996 non-null object
dtypes: object(20)
memory usage: 85.9+ KB


(1000, 20)

Now I'll read back in my csv and use this for all analysis and manipulation

In [10]:
## This code is required to reload the article data from CSV
#Now read my CSV with articles into a new dataframe.
articlesdf = pd.read_csv('nytimes06262016.csv')

In [22]:
#Make sure reading the file back in didn't change anything.
articlesdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 0 to 999
Data columns (total 21 columns):
Unnamed: 0           1000 non-null int64
_id                  1000 non-null object
abstract             13 non-null object
blog                 1000 non-null object
byline               933 non-null object
document_type        1000 non-null object
headline             1000 non-null object
keywords             1000 non-null object
lead_paragraph       993 non-null object
multimedia           1000 non-null object
news_desk            996 non-null object
print_page           230 non-null float64
pub_date             1000 non-null object
section_name         996 non-null object
slideshow_credits    17 non-null object
snippet              1000 non-null object
source               999 non-null object
subsection_name      561 non-null object
type_of_material     1000 non-null object
web_url              1000 non-null object
word_count           996 non-null float64
dtypes: float64(2), int

Something did change!  There's an extra unnamed column at the beginning.  What's up with that?

In [23]:
articlesdf.head()

Unnamed: 0                       _id abstract blog  \
0           0  5770a4e538f0d8170405490b      NaN   []   
1           1  5770a44438f0d8170405490a      NaN   []   
2           2  5770a2e438f0d81704054909      NaN   []   
3           3  5770a00a38f0d81704054907      NaN   []   
4           4  57709b3938f0d81704054905      NaN   []   

                                              byline document_type  \
0  {'original': 'By REUTERS', 'person': [], 'orga...       article   
1  {'original': 'By REUTERS', 'person': [], 'orga...       article   
2  {'original': 'By THE ASSOCIATED PRESS', 'perso...       article   
3  {'original': 'By REUTERS', 'person': [], 'orga...       article   
4  {'original': 'By THE ASSOCIATED PRESS', 'perso...       article   

                                            headline keywords  \
0  {'print_headline': 'Bank of Japan Can Stand Pa...       []   
1  {'print_headline': "Japan PM Campaigns on 'Sta...       []   
2  {'print_headline': 'Police: Man Fatally Shoots...       []   
3  {'print_headline': 'Suicide Bombers Kill Six P...       []   
4  {'print_headline': 'Simone Biles Glides to 4th...       []   

                                      lead_paragraph multimedia    ...      \
0  The Bank of Japan does not need to expand mone...         []    ...       
1  Japanese Prime Minister Shinzo Abe, stumping f...         []    ...       
2  Police say they've arrested a man who fatally ...         []    ...       
3  A number of suicide bombers killed six people ...         []    ...       
4  The national titles are starting to run togeth...         []    ...       

  print_page              pub_date  section_name slideshow_credits  \
0        NaN  2016-06-26T23:59:31Z  Business Day               NaN   
1        NaN  2016-06-26T23:54:41Z         World               NaN   
2        NaN  2016-06-26T23:50:30Z          U.S.               NaN   
3        NaN  2016-06-26T23:36:57Z         World               NaN   
4        NaN  2016-06-26T23:17:13Z        Sports               NaN   

                                             snippet   source subsection_name  \
0  The Bank of Japan does not need to expand mone...  Reuters             NaN   
1  Japanese Prime Minister Shinzo Abe, stumping f...  Reuters    Asia Pacific   
2  Police say they've arrested a man who fatally ...       AP             NaN   
3  A number of suicide bombers killed six people ...  Reuters     Middle East   
4  The national titles are starting to run togeth...       AP        Olympics   

  type_of_material                                            web_url  \
0             News  http://www.nytimes.com/reuters/2016/06/26/busi...   
1             News  http://www.nytimes.com/reuters/2016/06/26/worl...   
2             News  http://www.nytimes.com/aponline/2016/06/26/us/...   
3             News  http://www.nytimes.com/reuters/2016/06/26/worl...   
4             News  http://www.nytimes.com/aponline/2016/06/26/spo...   

  word_count  
0        159  
1        425  
2         79  
3         71  
4        850  

[5 rows x 21 columns]

Ok, just an index.  Not sure exactly why that happened, but moving on.
Ok, what can I do with this?  Maybe some initial analysis of some basic things.

In [24]:
#Look at word count
articlesdf['word_count'].describe()

count     996.000000
mean      553.970884
std       608.327771
min         6.000000
25%       174.000000
50%       413.000000
75%       738.250000
max      7318.000000
Name: word_count, dtype: float64

In [11]:
#how many articles per section?
grp_section = articlesdf.groupby('section_name')
grp_section.size()

section_name
Arts                   25
Books                  20
Business Day           76
Corrections             1
Crosswords & Games      4
Fashion & Style        54
Food                    2
Job Market              2
Magazine               15
Movies                  6
Multimedia/Photos       1
N.Y. / Region          33
NYT Now                 3
Opinion                24
Paid Death Notices     63
Real Estate            19
Sports                231
Style                   4
T Magazine              1
Technology              4
The Upshot              1
Theater                 3
Today’s Paper           1
Travel                 13
U.S.                  161
World                 228
Your Money              1
dtype: int64

In [46]:
#bar plot
num_articles = grp_section.size()
num_articles.plot(kind='bar');

So more sports articles than others in the last month.  What about average word counts for each section?

In [26]:
grp_section['word_count'].mean()

section_name
Arts                   760.680000
Books                 1075.650000
Business Day           556.506667
Corrections            516.000000
Crosswords & Games     568.250000
Fashion & Style        518.129630
Food                  1050.000000
Job Market             693.500000
Magazine              1901.928571
Movies                 977.000000
Multimedia/Photos       14.000000
N.Y. / Region          830.636364
NYT Now                512.500000
Opinion                997.250000
Paid Death Notices     136.285714
Real Estate            649.222222
Sports                 530.519481
Style                 1359.750000
T Magazine              15.000000
Technology             557.250000
The Upshot            1041.000000
Theater                945.333333
Today’s Paper           41.000000
Travel                1125.076923
U.S.                   462.354037
World                  461.614035
Your Money             995.000000
Name: word_count, dtype: float64

So the style and magazine articles are the wordiest on average

In [45]:
#boxplot of wordcount
articlesdf.boxplot('word_count', 'section_name', vert=False);

In [27]:
#stats for each section
grp_section['word_count'].describe().unstack()

count         mean          std   min      25%     50%  \
section_name                                                                 
Arts                   25   760.680000   636.361449    87   170.00   586.0   
Books                  20  1075.650000   904.572262   174   675.25   961.5   
Business Day           75   556.506667   647.682613    35   227.50   405.0   
Corrections             1   516.000000          NaN   516   516.00   516.0   
Crosswords & Games      4   568.250000   538.077055    38   285.50   463.5   
Fashion & Style        54   518.129630   707.705862     6   173.00   222.5   
Food                    2  1050.000000   824.486507   467   758.50  1050.0   
Job Market              2   693.500000   403.757972   408   550.75   693.5   
Magazine               14  1901.928571  2310.843817   152   703.75   867.0   
Movies                  6   977.000000   673.748321   146   681.75   853.0   
Multimedia/Photos       1    14.000000          NaN    14    14.00    14.0   
N.Y. / Region          33   830.636364  1068.558486    18    29.00   600.0   
NYT Now                 2   512.500000   709.228102    11   261.75   512.5   
Opinion                24   997.250000   437.612122    13   787.25  1010.0   
Paid Death Notices     63   136.285714   152.805213     7    42.50    82.0   
Real Estate            18   649.222222   677.819215    13    19.25   512.5   
Sports                231   530.519481   314.738371    59   304.00   512.0   
Style                   4  1359.750000   916.124582    29  1165.25  1655.0   
T Magazine              1    15.000000          NaN    15    15.00    15.0   
Technology              4   557.250000   420.652965   216   253.50   442.5   
The Upshot              1  1041.000000          NaN  1041  1041.00  1041.0   
Theater                 3   945.333333   728.924093   144   633.50  1123.0   
Today’s Paper           1    41.000000          NaN    41    41.00    41.0   
Travel                 13  1125.076923   744.867378   302   545.00   889.0   
U.S.                  161   462.354037   344.307192    24   136.00   370.0   
World                 228   461.614035   439.865298    32   173.00   326.0   
Your Money              1   995.000000          NaN   995   995.00   995.0   

                        75%   max  
section_name                       
Arts                1106.00  2237  
Books               1203.00  4594  
Business Day         677.50  5018  
Corrections          516.00   516  
Crosswords & Games   746.25  1308  
Fashion & Style      530.75  3565  
Food                1341.50  1633  
Job Market           836.25   979  
Magazine            1366.75  7318  
Movies              1140.50  2149  
Multimedia/Photos     14.00    14  
N.Y. / Region        887.00  3649  
NYT Now              763.25  1014  
Opinion             1230.50  2039  
Paid Death Notices   191.00   924  
Real Estate         1117.50  2356  
Sports               708.00  2806  
Style               1849.50  2100  
T Magazine            15.00    15  
Technology           746.25  1128  
The Upshot          1041.00  1041  
Theater             1346.00  1569  
Today’s Paper         41.00    41  
Travel              1537.00  2995  
U.S.                 691.00  2126  
World                595.25  3175  
Your Money           995.00   995

In [28]:
#how many by subsection
grp_section_subsection = articlesdf.groupby(['section_name','subsection_name'])
grp_section_subsection.size()

section_name     subsection_name   
Arts             Art & Design            2
                 Dance                   3
                 International Arts      1
                 Music                   5
                 Television              6
Books            Book Review            20
Business Day     DealBook                4
                 Economy                 1
Fashion & Style  Men's Runway            2
                 Men’s Style             2
                 Weddings               40
Opinion          Sunday Review          17
Sports           Auto Racing             5
                 Baseball               39
                 Cricket                 4
                 Cycling                 1
                 Golf                   22
                 Hockey                  4
                 Olympics               37
                 Pro Basketball          3
                 Pro Football            1
                 Rugby                  13
                 S

That doesn't include the sections without subsections.
Lots of Weddings- fashions, socer, and Europe artcles.  Europe makes sense due to the Brexit stuff.
Anyway, I'm not going to repeat all of homework 2 here :)
I'll move on to some other stuff.

In [12]:
#look for articles on certain topics
Brexitarticles = articlesdf[articlesdf.headline.str.contains("Brexit")]
len(Brexitarticles)


80

In [13]:
#how about our presidential candidates?
Trumparticles = articlesdf[articlesdf.headline.str.contains("Trump")]
len(Trumparticles)


21

In [14]:
Clintonarticles = articlesdf[articlesdf.headline.str.contains("Clinton")]
len(Clintonarticles)

6

In [15]:
Sandersarticles = articlesdf[articlesdf.headline.str.contains("Sanders")]
len(Sandersarticles)

1

In [16]:
Yankeesarticles = articlesdf[articlesdf.headline.str.contains("Yankees")]
len(Yankeesarticles)

2

This is where I was going to try to use the NYT Semantic API.
Unfortunately I couldn't get the API calls to work.  I'm not sure I understand the structure of that API.  I think it contains information about the possible keyword types and keywords in articles, but I didn't understand how to use it from the documentation:
http://developer.nytimes.com/semantic_api.json
I'm going to mve on to NLTK - a Python natural language package

In [17]:
#initial imports for nltk
from nltk import tokenize
from nltk.tokenize import PunktSentenceTokenizer


## Use the download command below to load nltk packages and corpus if not working:
May need to download:
from Corpora:
-Movie reviews
-Treebank
-words

from models:
-averaged perceptron tagger
-ACE named entity chunker
-punkt
-tagsets
-universal_tagset


In [87]:
#this will open up a downloader - only use this if code below is not working.
nltk.download()

showing info http://www.nltk.org/nltk_data/


True

In [18]:
#first going to try a few available nltk functions.
#Need to work on sentences - so I decided to try using article snippets for that.
testsentence = Brexitarticles.snippet[0]
testsentence


"The Bank of Japan does not need to expand monetary stimulus immediately if the market turbulence caused by Britain's vote to leave the European Union proves temporary, a former central bank executive said on Monday."

In [65]:
#tolkenize is a function that separates sentences into structure.
#It creates a list of words but also each token can be tagged with more information such as part of speech
tokens = nltk.word_tokenize(testsentence)
tokens

['The',
 'Bank',
 'of',
 'Japan',
 'does',
 'not',
 'need',
 'to',
 'expand',
 'monetary',
 'stimulus',
 'immediately',
 'if',
 'the',
 'market',
 'turbulence',
 'caused',
 'by',
 'Britain',
 "'s",
 'vote',
 'to',
 'leave',
 'the',
 'European',
 'Union',
 'proves',
 'temporary',
 ',',
 'a',
 'former',
 'central',
 'bank',
 'executive',
 'said',
 'on',
 'Monday',
 '.']

In [20]:
#This function tags each token with part of speech
tagged = nltk.pos_tag(tokens)
tagged[0:6]

[('The', 'DT'),
 ('Bank', 'NNP'),
 ('of', 'IN'),
 ('Japan', 'NNP'),
 ('does', 'VBZ'),
 ('not', 'RB')]

In [23]:
# this function draws a parse tree of the sentences' tokens
entities = nltk.chunk.ne_chunk(tagged[0:6])
entities

In [22]:
#from nltk.corpus import treebank #didn't end up using this function

In [31]:
nltk.data.load

<function nltk.data.load>

Now I'm going to train a Naive Bayes Classifier using a NLTK corpus and then ry to use that classifier to determine
if some of my articles get rated as 'positive' or 'negative'.
The next several code chunks are mostly copied from:
http://streamhacker.com/2010/05/10/text-classification-sentiment-analysis-naive-bayes-classifier/
This code uses movie reviews that are either positive or negative to train this model to identify
positive or negative words.
Finally, I'll use my trained classifier on some sentences from the NY times articles.

In [47]:
import nltk.classify.util
from nltk.classify import NaiveBayesClassifier
from nltk.corpus import movie_reviews

In [48]:
#this structure is requird for the classifier
def word_feats(words):
    return dict([(word, True) for word in words])

In [49]:
negids = movie_reviews.fileids('neg')
posids = movie_reviews.fileids('pos')

In [50]:
negfeats = [(word_feats(movie_reviews.words(fileids=[f])), 'neg') for f in negids]
posfeats = [(word_feats(movie_reviews.words(fileids=[f])), 'pos') for f in posids]

In [55]:
negcutoff = len(negfeats)*3//4
poscutoff = len(posfeats)*3//4
len(negfeats)*3//4 #making sure this comes out as an integer

750

In [63]:
trainfeats = negfeats[:negcutoff] + posfeats[:poscutoff]
testfeats = negfeats[negcutoff:] + posfeats[poscutoff:]
print ('train on 1500 instances, test on 500 instances' )
type(negfeats) #figuring out the object type rquired for classification.

train on 1500 instances, test on 500 instances


list

In [57]:
#train the classifier and show the words with the highest positve or negative rating.
classifier = NaiveBayesClassifier.train(trainfeats)
print ('accuracy:')
nltk.classify.util.accuracy(classifier, testfeats)
classifier.show_most_informative_features()

accuracy:
Most Informative Features
             magnificent = True              pos : neg    =     15.0 : 1.0
             outstanding = True              pos : neg    =     13.6 : 1.0
               insulting = True              neg : pos    =     13.0 : 1.0
              vulnerable = True              pos : neg    =     12.3 : 1.0
               ludicrous = True              neg : pos    =     11.8 : 1.0
             uninvolving = True              neg : pos    =     11.7 : 1.0
                  avoids = True              pos : neg    =     11.7 : 1.0
              astounding = True              pos : neg    =     10.3 : 1.0
             fascination = True              pos : neg    =     10.3 : 1.0
                 idiotic = True              neg : pos    =      9.8 : 1.0


In [70]:
#Now to try my trained classifer on my test article sentence tokens from above
classifier.classify(word_feats(tokens))

'pos'

So it looked at words in the article snippet, compared them with the trained classifier and determined
that the snipet was positive.
Now that I am able to classify a sentence, I'm going to try this on the NY Times articles on Trump
that I pulled from the API.  I'll use a loop to do this.

In [86]:
#this will loop through the articles on Trump and display 'pos' or 'neg' for each article snippet.
for index, row in Trumparticles.iterrows():
    sentencei = row['snippet']
    tokensi = nltk.word_tokenize(sentencei)
    classifyi = classifier.classify(word_feats(tokensi))
    print(sentencei)
    print (classifyi)
    print ('\n')

Hillary Clinton's presidential campaign accused Donald Trump on Sunday of caring more about how Britain's historic vote to leave the European Union would benefit his financial bottom line than how it would impact the U.S. economy.
neg


Republicans are sprinting to shape up Donald Trump's presidential campaign before the party's national convention in three weeks, even as leading members of the party carry a deep antipathy or outright opposition to his claim on the GOP nomination.
pos


At the heart of the campaign that led Britain to vote to leave the European Union was a desire to regain independence lost amid a globalized world. It's the same kind of feeling that Donald Trump rode to become the presumptive Republican nominee in...
pos


Here’s what you need to know about the week’s top stories.
neg


Several enterprises stamped with the Trump brand have been accused of preying upon desperation, inexperience or vanity.
neg


Touring one of his golf resorts, Mr. Trump spoke on the Bre

So mostly positive according to my trained classifier.
I'm going to have to stop here so I can turn in this project.

I would have liked to have dived deeper into NLTK and natural language processing, but ran out of time to do much more.
I spent too much time trying to figure out why my api calling code wasn't working correctly.
But I did eventually work out how to connect and interact with the NY times API, load article data into a dataframe, and do some analysis including the use of a sentiment analyzing classifier.